# Retail Demo Store - Personalized Offers Workshop - Lab 8

In this module we're going to be adding another sample use case of [Amazon Personalize](https://aws.amazon.com/personalize/): selecting offers or coupons to be providing to users. You might imagine these offers to be of the following kind:

 - Log in to our online store to get 10% off any product!
 - Enter PERFECTHAIR to get 3 for 2 on hair products online only! This coupon expires tomorrow!
 - and so forth.
 
In this notebook we'll ask Amazon Personalize to select a coupon for us that is likely to result in a conversion for the given user. We are going to ask Amazon Personalize to choose between the following coupons:

 - Enter AWS25 to earn a 25% discount on any product in store today!
 - Enter AMAZON20 to earn a 20% discount on any product in store today!
 - Enter PERSONALIZE15 to earn a 15% discount on any product in store today!
 - Enter SAGEMAKER10 to earn a 10% discount on any product in store today!
 - Enter SALE5 to earn a 5% discount on any product in store today!
 - No offer given.
 
One issue that this particular scenario brings to the fore is that personalization as generally practiced currently is designed to maximise conversions but choosing different offers for a user results in different costs to us.
A relatively new feature of Amazon Personalize is [the use of **additional objective functions**](https://docs.aws.amazon.com/personalize/latest/dg/optimizing-solution-for-objective.html).

With this feature, we can train Amazon Personalize to produce recommendations to maximise any objective we want - such as return on investment, lifetime value, and so forth.

Alternatively, we can incorporate some decision science into our systems ourselves. In the current context this might mean:

 - Limiting the number of certain types of offers to assign to users (for example, we might allow Personalize to select users for offers rather than selecting offers for users). In the current scenario, however, we want to ensure that every user has one offer.
 - Using the "contextual metadata" facility of Amazon Personalize to estimate the likely change in behaviour of users under different offer contexts. See the [personalize discounts](./Lab-04-(Core)-Evaluate-recommendations.ipynb) section in an earlier notebook for an idea about how such an approach may work. 
 - Using the scores provided by Personalize as probabilities as part of a decision system where the decision system chooses offers based on expected marginal return or other decision variables such as available coupons and take-up rate. For more about recommendation scores see this [blog article](https://aws.amazon.com/blogs/machine-learning/introducing-recommendation-scores-in-amazon-personalize/). We explore this option more in the present notebook in an empirical way, so that you can see for yourself what kind of scores Amazon Personalize gives back, and how they may be of use.


### Simulation 

We are going to simulate a simple dataset on which to train Amazon Personalize so that you can see how such a campaign may be trained with Amazon Personalize. Optionally, you can add skew to the data as might happen when certain items are much more valuable than others, so that we can explore how to deal with it via the balancing approaches discussed above.

Recommended Time: 2 Hours

## Setup

To get started, we need to perform a bit of setup. Walk through each of the following steps to configure your environment to interact with the Amazon Personalize Service.

### Import Dependencies and Setup Boto3 Python Clients

Throughout this workshop we will need access to some common libraries and clients for connecting to AWS services. We also have to retrieve Uid from a SageMaker notebook instance tag.

In [ ]:
# Import Dependencies

import boto3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import requests
import csv
import sys
import botocore
import uuid
import random
from collections import defaultdict

from packaging import version
from random import randint
from botocore.exceptions import ClientError

%matplotlib inline

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')

with open('/opt/ml/metadata/resource-metadata.json') as f:
  data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
        break

### Configure Bucket and Data Output Location

We will be configuring some variables that will store the location of our source data. When the Retail Demo Store stack was deployed in this account, an S3 bucket was created for you and the name of this bucket was stored in Systems Manager Parameter Store. Using the Boto3 client we can get the name of this bucket for use within our Notebook.

In [ ]:
response = ssm.get_parameter(
    Name='retaildemostore-stack-bucket'
)

bucket = response['Parameter']['Value']
interactions_filename = "offer_interactions.csv"  # This is where we will store our simulated data set

print('Bucket: {}'.format(bucket))

## Get, Prepare, and Upload User, Product, and Interaction Data

Amazon Personalize provides predefined recipes, based on common use cases, for training models. A recipe is a machine learning algorithm that you use with settings, or hyperparameters, and the data you provide to train an Amazon Personalize model. The data you provide to train a model are organized into separate datasets by the type of data being provided. A collection of datasets are organized into a dataset group. The three dataset types supported by Personalize are items, users, and interactions. Depending on the recipe type you choose, a different combination of dataset types are required. For all recipe types, an interactions dataset is required. Interactions represent how users interact with items. For example, viewing a product, watching a video, listening to a recording, or reading an article. For this workshop, we will be using a recipe that supports all three dataset types.

When we deployed the Retail Demo Store, it was deployed with an initial seed of fictitious User data as well as Offers. We will use this data to train a model, or solution, in the Amazon Personalize service which will be used to serve offer recommendations, related items,  and to rerank product lists for our users. The User and Product data can be accessed from the Retail Demo Store's Users and Offers microservices, respectively. We will access our data through microservice data APIs, process the data, and use it as the basis of a simualation of user-offer interactions.

Although Amazon Personalize can do a better job if we provide it with information ("metadata") about the users and items we are providing it with interactions data for, we do not focus on that capability here, and generate and upload only the interactions dataset. We upload it as CSV to S3. Once our dataset is in S3, we can import it into the Amazon Personalize service.

Let's get started.

### Get Offers Service Instance

We will be pulling our offers data from the Offers Service that was deployed in an Amazon Elastic Container Service as part of the Retail Demo Store. To connect to this service we will use [AWS Cloud Map](https://aws.amazon.com/cloud-map/)'s Service Discovery to discover an instance of the Product Service running in ECS, and then connect directly to that service instances to access our data.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='offers',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

offers_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Offers Service Instance IP: {}'.format(offers_service_instance))

#### Download and Explore the Offers Dataset

In [ ]:
response = requests.get('http://{}/offers'.format(offers_service_instance))
offers = response.json()
print(json.dumps(offers, indent=4))

Let us convert it to a Pandas dataframe for ease of exploration.

In [ ]:
offers_df = pd.DataFrame(offers['tasks'])
pd.set_option('display.max_rows', 5)
offers_df

In this example we do not use Amazon Personalize's ability to utilise user or item metadata. So we have no need to upload this data for Amazon Personalize - instead we will use it as the basis of generating user-offer item interactions below.

### Get Users Service Instance

We will be pulling some User data from the [Users Service](https://github.com/aws-samples/retail-demo-store/tree/master/src/users) that is deployed as part of the Retail Demo Store - we mostly just need available user IDs as we are not making use in this workshop of user or item metadata. To connect to this service we will use Service Discovery to discover an instance of the User Service, and then connect directly to that service instance to access our data.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

#### Download and Explore the Users Dataset

In [ ]:
response = requests.get('http://{}/users/all?count=10000'.format(users_service_instance))
users = response.json()
print(json.dumps(users[:2], indent=2))

Let us convert it to a Pandas dataframe for ease of exploration.

In [ ]:
users_df = pd.DataFrame(users)
# Remove any users without a persona or gender (i.e. created in web UI)
users_df = users_df[(users_df['persona'].str.strip().astype(bool)) | (users_df['gender'].str.strip().astype(bool))]
pd.set_option('display.max_rows', 5)
users_df


In this example we do not use Amazon Personalize's ability to utilise user or item metadata. So we have no need to upload this data for Amazon Personalize - instead we will use it as the basis of generating user-offer item interactions below.

### Create User-Offers Interactions Dataset

To mimic user behavior, we will be generating a new dataset that represents user interactions with items - in this use case, items consist of coupon offers to the users that users have made use of.

We do not consider potential relations between coupons and product context though this is very easy to do in Amazon Personalize using "contextual metadata". It is also possible to incorporate different "event types" to account for e.g. the difference between clicking on a coupon offer, adding it to a shopping cart, or redeeming it.


In the below parameters we determine the size of the dataset (we simulate exactly 3 offer interactions per user) as well as its date range. We keep this particular simulation deterministic so that we can recreate exactly the same dataset again. Also, we provide the option of making the dataset inbalanced - set the `INBALANCED` option to simulate different take up of coupons - e.g. if some items are vastly better at driving conversion than others (for example, when comparing a 5% to 25% discount) - we want to show later that it is possible wrap Personalize to take into account other business logic - for example, that some items are more profitable to business than others when they do experience conversions.

In [ ]:

INBALANCED = False
NUM_INTERACTIONS_PER_USER = 3

FIRST_TIMESTAMP = 1591803782  # 2020-06-10, 18:43:02
LAST_TIMESTAMP = 1599579782  # 2020-09-08, 18:43:02
RANDOM_SEED = 1

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


In [ ]:

def generate_data(interactions_filename, users_df, offers_df):
    """Script for writing to a file simulated user-offer interactions"""
    
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    num_users = users_df.shape[0]
    num_interactions = NUM_INTERACTIONS_PER_USER * num_users

    if INBALANCED:
        # We may wish to assume probability is proportional to ID to show off how we can add
        # business logic around Personalize
        offer_probs = offers_df.id.values.astype(float)
    else:
        # Or we can work around inbalance at the data munging stage
        offer_probs = np.ones(len(offers_df.id.values), dtype=float)

    # Normalise so that we have probabilities
    offer_probs = offer_probs / offer_probs.sum()

    # generate timestamps
    time_between_events = (LAST_TIMESTAMP - FIRST_TIMESTAMP) / num_interactions
    timestamps = np.arange(FIRST_TIMESTAMP, LAST_TIMESTAMP, time_between_events).astype(int)
    # pre-shuffle them as we will be using them as a randomising key when we sort by timestamp
    np.random.shuffle(timestamps)

    # generate all users Ids
    sample_user_ids = np.tile(users_df['id'].values.astype(int), NUM_INTERACTIONS_PER_USER)
    # only one event type
    event_type = ['OfferConverted'] * num_interactions  # Only one event type

    # we sort it to ensure there is a correlation between user ID and offer ID.
    sampled_offers = sorted(np.random.choice(offers_df.id.values, num_interactions, p=offer_probs))

    interactions_df = pd.DataFrame({'ITEM_ID': sampled_offers,
                                    'USER_ID': sample_user_ids,
                                    'EVENT_TYPE': event_type,
                                    'TIMESTAMP': timestamps})

    # by sorting by timestamp, other elements get shuffled
    interactions_df = interactions_df.sort_values('TIMESTAMP')

    with open(interactions_filename, 'w') as outfile:
        interactions_df.to_csv(outfile, index=False)

    globals().update(locals())  # This can be used for inspecting in console after script ran or if run with ipython.
    print('Generation script finished - created offers dataset')



In [ ]:
# In this cell, we run the script.
generate_data(interactions_filename, users_df, offers_df)

#### Open and Explore the Interactions Dataset

In [ ]:
!head "$interactions_filename"

In [ ]:
interactions_df = pd.read_csv(interactions_filename)
interactions_df

In [ ]:
interactions_df.ITEM_ID.value_counts().sort_index()

In [ ]:
interactions_df.ITEM_ID.value_counts().sort_index().plot.bar()
plt.title('Distribution of interactions by offer ID')
plt.xlabel('Offer ID')
plt.ylabel('Interaction count')
plt.show()

In [ ]:
interactions_df[['ITEM_ID', 'USER_ID']].plot.scatter(x='USER_ID', y='ITEM_ID', alpha=0.01)
plt.title('Item vs User ID correlation scatter plot')
plt.show()

#### Prepare and Upload Data

We push the data to an S3 bucket for Personalize to grab it from for training.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)
bucket, interactions_filename

## Configure Amazon Personalize

Now that we've prepared our dataset and uploaded it to S3 we'll need to configure the Amazon Personalize service to understand our data so that it can be used to train models for generating recommendations.

If Personalize auto-deployment was enabled when you deployed the demo, the below logic was run already by polling AWS Lambda function whose code is in the file `src/aws-lambda/personalize-pre-create-resources/personalize-pre-create-resources.py`. If you did not enable auto-deployment, you can run the code below to train your Personalize campaigns.

### Create Schemas for Dataset

Amazon Personalize requires a schema for its dataset so it can map the columns in our CSVs to fields for model training. Each schema is declared in JSON using the [Apache Avro](https://avro.apache.org/) format.

Let's define and create a schema in Personalize for our interactions dataset.

#### Interactions Dataset Schema

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

try:
    create_schema_response = personalize.create_schema(
        name = "retaildemooffers-schema-interactions",
        schema = json.dumps(interactions_schema)
    )
    interactions_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this schema, seemingly')
    schemas = personalize.list_schemas()['schemas']
    for schema_response in schemas:
        if schema_response['name'] == "retaildemooffers-schema-interactions":
            interactions_schema_arn = schema_response['schemaArn']


### Create and Wait for Dataset Group

Next we need to create the dataset group that will contain our three datasets.

#### Create Dataset Group

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = 'retaildemooffers-notebook'
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

print(f'DatasetGroupArn = {dataset_group_arn}')

#### Wait for Dataset Group to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Create Interactions Dataset

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "retaildemooffers-notebook-INTERACTIONS",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Import Datasets to Personalize

Up to this point we have generated CSVs containing data for our users, items, and interactions and staged them in an S3 bucket. We also created schemas in Personalize that define the columns in our CSVs. Then we created a datset group and a dataset in Personalize that will receive our data. In the following steps we will create an import job with Personalize that will import the dataset from our S3 bucket into the service.

### Inspect permissions

By default, the Personalize service does not have permission to acccess the data we uploaded into the S3 bucket in our account. In order to grant access to the  Personalize service to read our CSVs, we need to set a Bucket Policy and create an IAM role that the Amazon Personalize service will assume.

The deployment process for the Retail Demo Store has already setup these resources for you. However, let's take a look at the bucket policy and IAM role to see the required permissions.

We'll start by displaying the bucket policy in the S3 staging bucket where we uploaded the CSVs. Note the service principal of `personalize.amazonaws.com` and the actions allowed on the staging bucket. The `s3:GetObject` is needed for import jobs to allow Personalize to read objects from the bucket and the `s3:PutObject` is used for export jobs, batch inference jobs, and batch segment jobs to allow Personalize to write output files to the bucket. The `s3:ListBucket` action allows Personalize to list the contents of a folder.

In [ ]:
s3 = boto3.client("s3")

response = s3.get_bucket_policy(Bucket = bucket)
print(json.dumps(json.loads(response['Policy']), indent=2))

Next, let's look at the IAM role that Personalize will need to assume to access the S3 bucket. Again, this role was created for you during the Retail Demo Store deployment. We'll start by inspecting the role itself. Notice the same service principal as the bucket policy.

In [ ]:
iam = boto3.client("iam")

role_name = Uid+"-PersonalizeS3"

response = iam.get_role(RoleName = role_name)
role_arn = response['Role']['Arn']
print(json.dumps(response['Role'], indent=2, default = str))

Finally, we'll get the inline policy named `BucketAccess` that has the same S3 permissions as the bucket policy.

In [ ]:
response = iam.get_role_policy(RoleName = role_name, PolicyName = 'BucketAccess')
print(json.dumps(response, indent=2))

### Create Import Jobs

With the permissions in place to allow Personalize to access our CSV files, let's create three import jobs to import each file into its respective dataset. Each import job can take several minutes to complete so we'll create all three and then wait for them all to complete.

#### Create Interactions Dataset Import Job

In [ ]:
interactions_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "retaildemooffers-dataset-interactions-import-job",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = interactions_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(interactions_create_dataset_import_job_response, indent=2))

### Wait for Import Jobs to Complete

It will take 10-15 minutes for the import jobs to complete, while you're waiting you can learn more about Datasets and Schemas here: https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html


#### Wait for Items Import Job to Complete

In [ ]:
%%time

job_arn = interactions_dataset_import_job_arn

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = job_arn
    )
    status = import_job_response["datasetImportJob"]['status']

    if status == "ACTIVE":
        print(f'Import job {job_arn} successfully completed')
        break
    elif status == "CREATE FAILED":
        print(f'Import job {job_arn} failed')
        if import_job_response.get('failureReason'):
            print('   Reason: ' + import_job_response['failureReason'])
        break
    
    time.sleep(15)


## Create Solution

We can now turn to training a model to predict offers for a user. This involves training a model involves creating a Solution and Solution Version.

We have the choice between using the personalized recommendations and personalized ranking recipe - both of these return a set of scores, the main difference being that for the latter a list of items is provided. With these scores, it is possible to be more flexible with incorporating postprocessing logic. For more details: https://aws.amazon.com/blogs/machine-learning/introducing-recommendation-scores-in-amazon-personalize/

These two recipes are as below:

In [ ]:
recommend_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"
ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"
recommend_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

offer_recipe_arn = recommend_recipe_arn

These recipes also have different internal implementations and the way they generate scores can differ.

### Create Solutions and Solution Versions

With our recipes defined, we can now create our solution (training config) and solution version (trained model).

#### Create Offer Recommendation Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "retaildemo-offers-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = offer_recipe_arn
)

offers_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Offer Recommendation Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = offers_solution_arn
)

offers_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))


### Wait for Solution Version to Complete

Here we wait for the solution version to be created (training to finish).

#### Wait for Offers Solution Version to Have ACTIVE Status

In [ ]:
%%time

soln_ver_arn = offers_solution_version_arn

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    soln_ver_response = personalize.describe_solution_version(
        solutionVersionArn = soln_ver_arn
    )
    status = soln_ver_response["solutionVersion"]["status"]

    if status == "ACTIVE":
        print(f'Solution version {soln_ver_arn} successfully completed')
        break
    elif status == "CREATE FAILED":
        print(f'Solution version {soln_ver_arn} failed')
        if soln_ver_response.get('failureReason'):
            print('   Reason: ' + soln_ver_response['failureReason'])
        break
        
    time.sleep(15)


### Offline Metrics for Solution Version

Let's retrieve the metrics for the solution version we just created.

The easiest of these to understand is the "mean reciprocal rank at 25" (`mean_reciprocal_rank_at_25`). It is the mean (over test data) reciprocal of the rank of the actual next item (offer) a user interacted with from the 1st 25 items returned by the recommender.

The metrics are described in more detail [here](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html).

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = offers_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Create Campaigns


We can get batch recommendations from our trained solution but we may want to grab recommendations from our trained model in real-time. Creating a campaign is equivalent to deploying our trained model so that it gives us a real-time endpoint.

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "retaildemooffers-personalization",
    solutionVersionArn = offers_solution_version_arn,
    minProvisionedTPS = 1
)

offers_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for campaign to Have ACTIVE Status

In [ ]:
%%time

campaign_arn = offers_campaign_arn

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = campaign_response["campaign"]["status"]

    if status == "ACTIVE":
        print(f'Campaign {campaign_arn} successfully completed')
        break
    elif status == "CREATE FAILED":
        print(f'Campaign {campaign_arn} failed')
        if campaign_response.get('failureReason'):
            print('   Reason: ' + campaign_response['failureReason'])
        break
    time.sleep(15)


## Test Campaigns

Now that our campaign has been fully created, let's test it and evaluate it.

#### Select a user

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change. Let's print out the user details - in our situation it is only the user ID that might be relevant as we only used the user ID and no other metadata about the user to alter how interactions are generated.

In [ ]:
user_id = 256

response = requests.get('http://{}/users/id/{}'.format(users_service_instance, user_id))
user = response.json()
print(json.dumps(user, indent=4, sort_keys=True))

#### Get Offer Recommendations for User

Now let's call Amazon Personalize to get recommendations for our user from the offers recommendations campaign.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = offers_campaign_arn,
    userId = str(user_id),
    numResults = 5
)

item_list = get_recommendations_response['itemList']

In [ ]:
print(json.dumps(item_list, indent=4))

Notice that in this response we have `score` field returned with each `itemId`.
For all recipes except SIMS and Popularity-Count, Personalize
[calculates a score](https://docs.aws.amazon.com/personalize/latest/dg/getting-real-time-recommendations.html)
for each recommended item **across all items in your dataset**.
Score values are between 0.0 and 1.0 and the sum of all scores across all items will total to 1.0.
Therefore, the absolute value of scores will be smaller for larger item catalogs.


### Evaluate results distribution

We are interested in seeing what distribution of scores items our recommender is returning
considering the dataset we provided Personalize above. Let us call the real-time recommender a few times in order to get a feel for the distribution. We can plot the distribution of scores and of top-ranked recommendations.

We can also provide a multiplier for each offer type that we multiply into the score. The multiplier can reflect business logic, such as relative expected return on each offer. Play with the multiplier calculated below to see different results.

In [ ]:
num_users_to_get_offers_for = 100
offers_df = offers_df.set_index('id', drop=False)  # Let us index our offers dataframe by offer ID.
# For example, we might like to weight inverse to ID. 
# You can play with these multipliers or try different schemes to get a feel for the range of possibilities here.
adjustment_multipliers = 1 / offers_df.id.astype(float) 

# Depending on whether we chose to use the ranking or user personalization recipe, 
# we get slightly different data back from the real-time campaign.
if offer_recipe_arn == recommend_recipe_arn:
    return_key = 'itemList'
elif offer_recipe_arn == ranking_recipe_arn:
    return_key = 'personalizedRanking'

# This is going to keep all the scores returned for further exploration.
all_user_scores = []
# After we apply our adjustment we get different scores - we can check how they look here.
all_adjusted_scores = []

sum_user_scores = defaultdict(float)
sum_adjusted_scores = defaultdict(float)

# We are going to plot as we go.
plt.figure(figsize=[15,5])
import time
for user_id in users_df.id.sample(n=num_users_to_get_offers_for):
    if offer_recipe_arn == recommend_recipe_arn:
        get_recommendations_response = personalize_runtime.get_recommendations(
           campaignArn = offers_campaign_arn,
           userId = str(user_id),
           numResults = offers_df.shape[0]
        )
    elif offer_recipe_arn == ranking_recipe_arn:
        get_recommendations_response = personalize_runtime.get_personalized_ranking(
            campaignArn = offers_campaign_arn,
            userId = str(user_id),
            inputList=list(offers_df.id.astype(str))
        )

    time.sleep(0.01)  # rate limit
    
    sum_scores = sum(float(item['score']) for item in get_recommendations_response[return_key])
    
    # Here we extract all the scores and put them into an offer: score dictionary.
    user_scores = {int(item['itemId']):float(item['score'])/sum_scores for item in get_recommendations_response[return_key]}
    
    # Here we see what would happen to our scores if we applied an adjustment multiplier.
    adusted_scores = {key: value * adjustment_multipliers[key] for key, value in user_scores.items()}

    for key in user_scores:
        sum_user_scores[key] += user_scores[key]
        sum_adjusted_scores[key] += adusted_scores[key]
    
    all_user_scores.append(user_scores)
    all_adjusted_scores.append(adusted_scores)

# Let us plot every single score received.
plt.plot(sum((list(user_scores.keys()) for user_scores in all_user_scores), []), 
         sum((list(user_scores.values()) for user_scores in all_user_scores), []), 
         'r.', label='All Scores from Personalize')
plt.plot(sum((list(adusted_scores.keys()) for adusted_scores in all_adjusted_scores), []), 
         sum((list(adusted_scores.values()) for adusted_scores in all_adjusted_scores), []), 
         'b.', label='All Adjusted Scores from Personalize')

    
# Turn the sums into averages so we can plot them against the individual score distribution.
all_user_sum = sum(sum_user_scores.values())
all_adjusted_sum = sum(sum_user_scores.values())
for key in sum_user_scores:
    sum_user_scores[key] = sum_user_scores[key] / all_user_sum
    sum_adjusted_scores[key] = sum_adjusted_scores[key] / all_adjusted_sum

# Let us plot the average scores, so we can see how well our adjustment technique is doing.
plt.plot(list(sum_user_scores.keys()), list(sum_user_scores.values()), 'm_', markersize=35, label='Mean Score from Personalize')
plt.plot(list(sum_adjusted_scores.keys()), list(sum_adjusted_scores.values()), 'g_', markersize=35, label='Mean Adjusted Score from Personalize')

plt.ylabel("Score")
plt.xlabel("Offer ID")
plt.legend()
plt.ticklabel_format(style='plain',axis='x',useOffset=False)
plt.title('Distribution of scores returned.')
plt.show()

Let us examine the distribution of scores using a histogram.

In [ ]:
fig, ax = plt.subplots(5, constrained_layout=True, figsize=[5,5])
plotdf = pd.DataFrame(all_user_scores)
bins = np.arange(0,0.4,0.001)
for offer_id in range(1,6):
    plotind = offer_id-1
    ax[plotind].set_xlim([0,0.4])
    plotdf[[offer_id]].plot.hist(bins=bins, alpha=0.5, ax=ax[offer_id-1])
    if offer_id != 5:
        ax[plotind].set_xticklabels([])        
    if offer_id != 3:
        ax[plotind].set_ylabel('')
    ax[plotind].set_title(f'Offer {offer_id}')
    ax[plotind].get_legend().remove()
        
fig.suptitle('Histograms of score for each offer ID')    
plt.show()

## The Location Services demo

A campaign similar to this personalized offers campaign is made use of in the Location Services demo; coupons are selected by Amazon Personalize and forwarded on to relevant customers using Amazon Pinpoint's segmenting, campaign, SMS and email capabilities as well as AWS API Gateway web push. These coupons provide potential users motivation to engage in the location-aware functionality in the Retail Demo Store and get in to the physical store itself for a truely local experience.

For more information on that demo, see "Retail Geofencing and Location-aware Personalization" in the Demo Guide. 

## Lab complete

Congratulations! You have completed the Retail Demo Store personalized offers lab.